In [11]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

In [12]:
yaml_path = os.path.join(os.getcwd(), 'data-yolo', 'data.yaml')

In [13]:
def train_yolov11(yaml_path, epochs=30, batch_size=16, pretrained_model=None, img_size=416, learning_rate=0.0008, optimizer="Adam", patience=7, device=0):
    """
    Train YOLOv11 on the prepared dataset, optionally starting from a pretrained model.
    
    Args:
        yaml_path (str): Path to the data.yaml file.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.
        pretrained_model (str): Path to a pretrained model to continue training. Default is None.
        img_size (int): Image size for training. Default is 416.
        learning_rate (float): Initial learning rate. Default is 0.0008.
        optimizer (str): Optimizer to use. Default is 'Adam'.
        patience (int): Number of epochs with no improvement after which training will be stopped. Default is 7.
        device (int): Device to use for training. Default is 0 (GPU).
    """
    # Ensure data.yaml exists
    if not os.path.exists(yaml_path):
        raise FileNotFoundError(f"data.yaml file not found: {yaml_path}")

    # Debug: Check if the dataset paths are correct
    print(f"Training with dataset located at: {yaml_path}")
    
    # Load YOLOv11 model (optionally using a pretrained model)
    model = YOLO(pretrained_model if pretrained_model else "yolo11s.pt")

    # Debug: Verify the model is loaded
    print(f"Model {pretrained_model if pretrained_model else 'yolo11s.pt'} loaded successfully.")
    
    # Train the model
    model.train(
        data=yaml_path,
        epochs=epochs,               # Use the provided epochs parameter
        batch=batch_size,            # Use the provided batch_size parameter
        imgsz=img_size,              # Use the provided img_size parameter
        lr0=learning_rate,           # Use the provided learning_rate parameter
        lrf=0.000001,                # Learning rate decay factor
        optimizer=optimizer,         # Use the provided optimizer parameter
        patience=patience,           # Use the provided patience parameter
        val=True,
        project="runs",
        name="yolov11_garbage_optimized",
        plots=True,
        device=device,               # Use the provided device parameter
        exist_ok=True                # This will overwrite the existing folder
    )

    print("YOLOv11 training complete.")
    
    # Save the model after training
    model.save("yolov11_garbage_detection_model.pt")
    print("Model saved as 'yolov11_garbage_detection_model.pt'.")

In [14]:
def evaluate_model(model, yaml_path):
    """
    Evaluate the model using mAP and TensorBoard.

    Args:
        model: Trained YOLOv11 model.
        dataset (dict): Dictionary containing dataset paths.

    Returns:
        dict: Evaluation metrics including mAP scores.
    """
    print("Evaluating model performance...")

    # Run validation
    results = model.val(
        data=yaml_path,
        project="runs",
        name="evaluation YOLOv11 Model",
        imgsz=416,
        split='test',
        save_json=True,
        plots=True,
        exist_ok=True  # This will overwrite the existing evaluation folder
    )

    # Extract metrics safely
    try:
        metrics = {
            "mAP@0.5": results.box.map50,      # mAP@0.5
            "mAP@0.5:0.95": results.box.map    # mAP@0.5:0.95
        }
        print(f"Evaluation metrics: {metrics}")
        return metrics
    except AttributeError as e:
        raise ValueError("The results object does not contain expected metrics.") from e

In [15]:
# Train the model
train_yolov11(
    yaml_path=yaml_path,
    epochs=250,
    batch_size=16,
    pretrained_model=None,
    img_size=416,
    learning_rate=0.0008,
    optimizer="Adam",
    patience=7,
    device=0
)

# Load the trained model
model = YOLO("yolov11_garbage_detection_model.pt")

# Evaluate model performance
metrics = evaluate_model(model, yaml_path)

Training with dataset located at: c:\Users\User\Desktop\Garbage-Classification\data-yolo\data.yaml
Model yolo11s.pt loaded successfully.
New https://pypi.org/project/ultralytics/8.3.68 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.67  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=c:\Users\User\Desktop\Garbage-Classification\data-yolo\data.yaml, epochs=250, time=None, patience=7, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=yolov11_garbage_optimized, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300

train: Scanning C:\Users\User\Desktop\Garbage-Classification\data-yolo\train\labels... 711 images, 0 backgrounds, 0 corrupt: 100%|██████████| 711/711 [00:00<00:00, 2120.47it/s]


train: New cache created: C:\Users\User\Desktop\Garbage-Classification\data-yolo\train\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 1610. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


KeyboardInterrupt: 